# LFPCA on Visual Grating
### Visual Grating Task
> Monkey was sitting with head fixed. His arm motion was also restrained. 
ECoG data and eye position were recorded. There was a monitor in front of the monkey. 
Grating pattern that moves in eight direction was presented on the screen. 
There was no fixation required. Blank and stimulus pattern were switched alternatively every 2 sec. 
Stimulus events can be decoded from 129th channel by analog value. 
ECoG data were sampled at 1KHz.

* Data Format

    A. ECoG_chN.mat
        ECoGData_chN: ECoG signal (μV) recorded from electrodeN (1‐128), sampled at 1kHZ.
        The Location of electrode is documented in "K2.png".

    B. ECoG_time.mat
        ECoGTime: ECoGTime is a one row-vector contains Time-stamps with the same length as ECoGData_chN.

    C. Event.mat
        EventData: EventData is a one row-vector contains analog input.
        -Analog input format
      Ain indicates event time.
      [   0- 100] : preparation time before experiment
      [ 300- 400] : ITI
      [ 650- 750] : Stimulus 1 ( 45 degree)
      [ 950-1050] : Stimulus 2 ( 90 degree)
      [1300-1400] : Stimulus 3 (135 degree)
      [1600-1700] : Stimulus 4 (180 degree)
      [1950-2050] : Stimulus 5 (225 degree)
      [2250-2350] : Stimulus 6 (270 degree)
      [2600-2700] : Stimulus 7 (315 degree)
      [2900-3000] : Stimulus 8 (360 degree)

    EventTime: EventTime is a one row-vector contains corresponding time-stamp.

In [1]:
import sys
# sys.path.append('/Users/Lauren/Voytek/spectralCV')
sys.path.append('/Users/SP/code/spectralCV')
sys.path.append('/Users/SP/code/neurodsp/')
sys.path.append('/Users/SP/code/fooof/')

In [2]:
%matplotlib inline

# imports
from scv_funcs import access_nt as asc
import numpy as np
import scipy as sp
import scipy.io as io
import scipy.signal as sig
import matplotlib.pyplot as plt
import pandas as pd
import h5py
import os
from scv_funcs import utils
CKEYS = plt.rcParams['axes.prop_cycle'].by_key()['color']

import neurodsp as ndsp
# from fooof import FOOOFGroup
from scv_funcs import lfpca

In [3]:
# data_path = "/Users/Lauren/Data/NeuroTycho/VisualGrating/20100723S1_VGT_K2_KazuhitoTakenaka-ToruYanagawa_mat_ECoG128-Event3/"
data_path = "/Volumes/VLABB/Neurotycho/VisualGrating/20100723S1_VGT_K2_KazuhitoTakenaka-ToruYanagawa_mat_ECoG128-Event3/"
# setting event data into pandas dataframe
matfile = io.loadmat(data_path + 'Event.mat', squeeze_me=True)
EventData = matfile['EventData']
EventIndex = matfile['EventIndex']

df = pd.DataFrame(data = [EventData, EventIndex]).T
df = df.rename(columns={0: 'EventData', 1: 'EventIndex'})

In [7]:
# blanks
ITI = df[(df['EventData'] >= 300) & (df['EventData'] <= 400)]

In [53]:
ITI_chunks = []

In [55]:
i = 0
counter = 0
while i < ITI.shape[0]-1:
    if ITI.iloc[[i]]['EventIndex'].values+1 == ITI.iloc[[i+1]]['EventIndex'].values:
        counter += 1
        i += 1
    else:
        ITI_chunks.append((i+1, counter))
        counter = 0
        i += 1

In [56]:
ITI_chunks

[(2032, 2031),
 (4033, 2000),
 (6034, 2000),
 (8035, 2000),
 (10036, 2000),
 (12037, 2000),
 (14039, 2001),
 (16040, 2000),
 (18042, 2001),
 (20044, 2001),
 (22046, 2001),
 (24048, 2001),
 (26050, 2001),
 (28052, 2001),
 (30054, 2001),
 (32056, 2001),
 (34058, 2001),
 (36060, 2001),
 (38062, 2001),
 (40097, 2034),
 (42131, 2033),
 (44166, 2034),
 (46168, 2001),
 (48170, 2001),
 (50172, 2001),
 (52174, 2001),
 (54176, 2001),
 (56177, 2000),
 (58178, 2000),
 (60179, 2000),
 (62180, 2000),
 (64181, 2000),
 (66183, 2001),
 (68184, 2000),
 (70185, 2000),
 (72187, 2001),
 (74187, 1999),
 (76188, 2000),
 (78189, 2000),
 (80190, 2000),
 (82191, 2000),
 (84193, 2001),
 (86195, 2001),
 (88197, 2001),
 (90199, 2001),
 (92201, 2001),
 (94203, 2001),
 (96205, 2001),
 (98223, 2017),
 (100258, 2034),
 (102293, 2034),
 (104294, 2000),
 (106296, 2001),
 (108298, 2001),
 (110300, 2001),
 (112302, 2001),
 (114303, 2000),
 (116304, 2000),
 (118306, 2001),
 (120308, 2001),
 (122310, 2001),
 (124311, 2000),

In [57]:
# getting session data
session = 0
chan = np.arange(1,129).tolist()
# chan = [1]
data = asc.get_ECoG(data_path, session, chan)

# ITI blank data
ITIdata = np.vstack((data[0][ITI['EventIndex']], data[1][ITI['EventIndex']]))
for c in range(2,128):
    ITIdata = np.vstack((ITIdata, data[c][ITI['EventIndex']]))

In [ ]:
saveout_path = "../results/nt_vg/"
lfpca_files = sorted([f for f in os.listdir(saveout_path) if 'lfpca_' in f])
# fooof_files = sorted([f for f in os.listdir(saveout_path) if 'fooof' in f])

In [ ]:
vg_lfpca_objs = np.empty((9,),dtype=object)

In [ ]:
data[0][ITI['EventIndex']

In [ ]:
# blanks
ITI = df[(df['EventData'] > 300) & (df['EventData'] < 400)]

# different stimulations 1 to 8 according to the boundary edges
edges = [(650,750), (950,1050), (1300,1400), (1600,1700), (1950,2050), (2250,2350), (2600,2700), (2900,3000)]
stims = np.empty((8,),dtype=object)
for i,edge in enumerate(edges):
    lower = edge[0]
    upper = edge[1]
    stims[i]= df[(df['EventData'] > lower) & (df['EventData'] < upper)]
    
# getting session data
session = 0
chan = np.arange(1,129).tolist()
# chan = [1]
data = asc.get_ECoG(data_path, session, chan)

# ITI blank data
ITIdata = np.vstack((data[0][ITI['EventIndex']], data[1][ITI['EventIndex']]))
for c in range(2,128):
    ITIdata = np.vstack((ITIdata, data[c][ITI['EventIndex']]))

In [ ]:
# example of using LFPCA
fs = 1000
analysis_param = {'nperseg': 1000,
                  'noverlap': 0,
                  'spg_outlierpct': 2.,
                  'max_freq':200}

vg_lfpca = lfpca.LFPCA(analysis_param)
vg_lfpca.populate_ts_data(ITIdata, fs)
vg_lfpca_objs[8] = vg_lfpca